In [1]:
import yaml
import pickle
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import scipy.stats as stats
import openai

In [ ]:
df = pd.read_json('../data/dataset-term-reward.json')
df.head()

In [3]:
with open('./openai-key.yaml') as f:    
    openai.api_key = yaml.safe_load(f)['key']

In [4]:
rand = stats.truncnorm(-1, 2, loc=8, scale=1)

In [5]:
answer_lt = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    text = row['text']
    prompt = f'다음 약관을 쉬운 말로 가장 핵심만 요약해서 풀어서 설명해줘.: \n{text}\n\n이 약관의 핵심 내용은 '
    for _ in range(10):
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo-16k',
                messages=[
                    {
                        'role': 'user',
                        'content': prompt
                    }
                ],
                max_tokens=300,
            )
        except:
            continue
        answer = response['choices'][0]['message']['content']
        break
    else:
        continue
    
    answer_lines = answer.splitlines()
    last_line = answer_lines[-1].strip()
    if (':' not in last_line or last_line[-1] == ':'):
        answer = '\n'.join(answer_lines[:-1])
    
    answer_lt.append({
        'row_no': row['row_no'],
        'summary2': answer,
        'reward': round(rand.rvs())
    })
    with open('./tmp_answer_lt.pkl', 'wb') as f:
        pickle.dump(answer_lt, f)

  0%|          | 0/500 [00:00<?, ?it/s]

In [14]:
df2 = pd.DataFrame(answer_lt)
len(df2)

483

In [15]:
df2.isna().sum()

row_no      0
summary2    0
reward      0
dtype: int64

In [16]:
len(df)

500

In [ ]:
df2.head()

In [17]:
total_df = pd.merge(df, df2, on='row_no').rename(columns={'reward_x': 'reward1', 'reward_y': 'reward2', 'summary': 'summary1'})

In [18]:
total_df.loc[total_df['summary2'] == '', 'reward2'] = np.nan
total_df.loc[total_df['summary2'] == '', 'summary2'] = np.nan
total_df.isna().sum()

row_no        0
text          0
summary1      0
reward1       0
summary2    109
reward2     109
dtype: int64

In [28]:
df2 = total_df[['text', 'summary2', 'reward2']]

In [29]:
len(df2)

483

In [30]:
df2 = df2.dropna()
len(df2)

374

In [35]:
df2 = df2.rename(columns={'summary2': 'summary', 'reward2': 'reward'})

In [41]:
#df = df.drop(columns=['row_no'])
#df2 = df2.drop(columns=['row_no'])
pd.concat([df, df2], axis=0).reset_index().to_json('../data/preprocessed/dataset-term-reward.json')

In [ ]:
df['summary2'] = pd.Series(answer_lt)
df['reward2'] = np.random.randint(7, 10, size=len(answer_lt))
df.head()

In [21]:
total_df.to_json('../data/preprocessed/dataset-term-reward.json')